**Preparing dataframe for analysis:**
- Goal: match the football rankings week-by-week data together by season, week, and team name. This will require (1) standardizing the team names and (2) standardizing the week names.

**Errors**
- Weeky by Week errors:
    - Does Georgia Southern Eagles have a week by week ranking -- they don't appear in the week by week rankings. Seems like quite a few do not have an associated state. (e.g., only 85 teams)
    - Purdue Boilermakers is not in the list
- Negative issues
    - Some weeks have negative values

In [2]:
import pandas as pd
import seaborn as sn

df_football = pd.read_csv('CollegeFootballRankings24to20.csv')
df_rankings = pd.read_csv('WeekByWeekRankings2020to2025.csv')
df_teams = pd.read_csv('2025 Teams.csv')

# Update Week Dataframe
df_rankings_cleaned = pd.merge(df_rankings, df_teams, how="left", on="Team")
print(f"There are {df_rankings_cleaned['team_and_mascot'].isnull().sum()} errors")
df_rankings_cleaned = df_rankings_cleaned.rename(columns={"year": "season", "Team":"state","Week" : "week","team_and_mascot" : "team"}) # Renamed
df_football = df_football.rename(columns={"Team" : "team"})
df_rankings_cleaned.to_csv("WeekbyWeekRankingsUpdated.csv")

# Merge the final Dataframe
df_analyze = pd.merge(df_football, df_rankings_cleaned, how = "left", on=["team","season","week"])
df_analyze.to_csv("Cleaned_Dataset.csv")




There are 2 errors


In [3]:
df_analyze.head()

,team,date,Team_id,opponent,win_loss,OT,OT_num,pass,rush,rec,...,CONF,SOR,FPI,AP/CFP,SOS,GC,AVGWP,AP_rank,state,code
0,Mississippi State Bulldogs,"Sat, Sep 26",344,LSU,W,False,NaN,623.0,34.0,183.0,...,SEC,70,52,--,4,61,90,NaN,NaN,NaN
1,UCF Knights,"Sat, Oct 17",2116,Memphis,L,False,NaN,601.0,78.0,175.0,...,American,56,22,--,68,18,12,NaN,NaN,NaN
2,Georgia Southern Eagles,"Sat, Oct 15",290,James Madison,W,False,NaN,578.0,20.0,164.0,...,Sun Belt,97,91,--,118,109,90,NaN,NaN,NaN
3,Western Kentucky Hilltoppers,"Fri, Dec 3",98,UTSA,L,False,NaN,577.0,15.0,179.0,...,CUSA,61,41,--,92,64,43,NaN,NaN,NaN
4,Ohio State Buckeyes,"Sat, Jan 1",194,Utah,W,False,NaN,573.0,83.0,347.0,...,Big Ten,8,3,6,27,4,8,7.0,Ohio State,194.0
